**Title:** Training state-of-the-art neural networks in the Microsoft Azure cloud<br/>
**Author:** Adrian Rosebrock, [PyImageSearch.com](https://www.pyimagesearch.com/)

The code associated with this notebook is part of my book, [*Deep Learning for Computer Vision with Python*](https://www.pyimagesearch.com/deep-learning-computer-vision-python-book/) where I demonstrate how to replicate state-of-the-art deep learning results on the large, 1.2+ million image ImageNet dataset. To learn more, please refer to the [book page](https://www.pyimagesearch.com/deep-learning-computer-vision-python-book/).

In [1]:
# import the necessary packages
from config import imagenet_squeezenet_config as config
import mxnet as mx
import json
import os

# initialize an arguments dictionary (used to replace argument parsing but
# keep the same code base)
args = {
    "checkpoints": "checkpoints",
    "prefix": "squeezenet",
    "epoch": 80
}

In [2]:
# load the RGB means for the training set
means = json.loads(open(config.DATASET_MEAN).read())

# construct the testing image iterator
testIter = mx.io.ImageRecordIter(
    path_imgrec=config.TEST_MX_REC,
    data_shape=(3, 227, 227),
    batch_size=config.BATCH_SIZE,
    mean_r=means["R"],
    mean_g=means["G"],
    mean_b=means["B"])

In [3]:
%%time
# load the checkpoint from disk
print("[INFO] loading model...")
checkpointsPath = os.path.sep.join([args["checkpoints"],
    args["prefix"]])
model = mx.model.FeedForward.load(checkpointsPath,
    args["epoch"])

[INFO] loading model...
CPU times: user 9.69 ms, sys: 0 ns, total: 9.69 ms
Wall time: 8.35 ms


/anaconda/envs/py35/lib/python3.5/site-packages/mxnet-0.12.0-py3.5.egg/mxnet/model.py:901: DeprecationWarning: mxnet.model.FeedForward has been deprecated. Please use mxnet.mod.Module instead.
  **kwargs)


In [4]:
%%time
# compile the model
model = mx.model.FeedForward(
    ctx=[mx.gpu(0)],
    symbol=model.symbol,
    arg_params=model.arg_params,
    aux_params=model.aux_params)

# make predictions on the testing data
print("[INFO] predicting on test data...")
metrics = [mx.metric.Accuracy(), mx.metric.TopKAccuracy(top_k=5)]
(rank1, rank5) = model.score(testIter, eval_metric=metrics)

# display the rank-1 and rank-5 accuracies
print("[INFO] rank-1: {:.2f}%".format(rank1 * 100))
print("[INFO] rank-5: {:.2f}%".format(rank5 * 100))

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:6: DeprecationWarning: mxnet.model.FeedForward has been deprecated. Please use mxnet.mod.Module instead.
  


[INFO] predicting on test data...
[INFO] rank-1: 56.86%
[INFO] rank-5: 79.38%
CPU times: user 5min 7s, sys: 33.8 s, total: 5min 40s
Wall time: 2min 21s
